In [ ]:
import anndata as ad
import networkx as nx
import scanpy as sc
import pandas as pd
import numpy as np
import scglue
from matplotlib import rcParams
import os
os.chdir('/lustre/scratch/kiviaho/spatac/integrations/tonsilatlas/actual_scatac_to_spatial')
date = '20221215'
multiple_experiments = False

In [ ]:
rna_name = 'actual_spatial_rna'
atac_name = 'simulated_spots_atac'

rna = ad.read_h5ad(rna_name+'.h5ad')
atac = ad.read_h5ad(atac_name + '.h5ad')

# Copy raw counts into X, only if 
# For single cell
#rna.X = rna.raw.X.copy()
#atac.X = atac.raw.X.copy()

# For simulated data
rna.layers['counts'] = rna.X.copy()
atac.layers['counts'] = atac.X.copy()

In [ ]:
scglue.data.get_gene_annotation(
    rna, gtf='../../../gencode.v42.annotation.gtf.gz',
    gtf_by="gene_name"
)

In [ ]:
# Drop unannotated genes:
rna = rna[:,~rna.var.index.duplicated(keep='first')]
rna = rna[:,rna.var.dropna(subset=['chrom','chromStart','chromEnd']).index]

In [ ]:
rna.var['chrom'] = rna.var['chrom'].astype(str).copy()
rna.var['chromStart'] = rna.var['chromStart'].astype(int).copy()
rna.var['chromEnd'] = rna.var['chromEnd'].astype(int).copy()

In [ ]:
sc.pp.highly_variable_genes(rna, n_top_genes=2000, flavor="seurat_v3",span=1)
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna)
sc.tl.pca(rna, n_comps=100, svd_solver="auto")

In [ ]:
# Drop peak locations with zero peaks
atac = atac[:,~(atac.X.sum(axis=0)==0)]

# Extract coordinates
atac.var.rename(columns={'seqnames':'chrom','start':'chromStart','end':'chromEnd'},inplace=True) # just rename existing columns !b

""" split = atac.var_names.str.split(r"[:-]")
stacked_split = np.vstack(split).astype(str)
var_metadata = atac.var.copy()
chrom_info = pd.DataFrame({'chrom':stacked_split[:,0],
'chromStart':stacked_split[:,1],
'chromEnd':stacked_split[:,2]})
chrom_info.index = var_metadata.index
var_metadata = pd.concat([var_metadata,chrom_info],axis=1) """

In [ ]:

# Moved guidance graph construction to BEFORE lsi calculation, since it should be using HVGs (which it gets from the RNA)
guidance = scglue.genomics.rna_anchored_guidance_graph(rna, atac)
scglue.graph.check_graph(guidance, [rna, atac])
nx.write_graphml(guidance, "guidance_graph_"+atac_name+"_"+rna_name+"_"+date+".graphml.gz")

In [ ]:
sub_atac = atac[:5]
sub_rna = rna[:5]
# For some weird reason atac var column names revert back to the old ones. Correct them
sub_atac.var.rename(columns={'seqnames':'chrom','start':'chromStart','end':'chromEnd'},inplace=True) # 
sub_atac.var['chrom'] = sub_atac.var['chrom'].astype(str).copy()
sub_atac.var['chromStart'] = sub_atac.var['chromStart'].astype(int).copy()
sub_atac.var['chromEnd'] = sub_atac.var['chromEnd'].astype(int).copy()

In [ ]:

# Moved guidance graph construction to BEFORE lsi calculation, since it should be using HVGs (which it gets from the RNA)
guidance = scglue.genomics.rna_anchored_guidance_graph(sub_rna, sub_atac)
scglue.graph.check_graph(guidance, [sub_rna, sub_atac])
nx.write_graphml(guidance, "guidance_graph_"+atac_name+"_"+rna_name+"_"+date+".graphml.gz")

In [ ]:
atac.var = sub_atac.var.copy()

In [ ]:
nx.write_graphml(guidance, "guidance_graph_"+atac_name+"_"+rna_name+"_"+date+".graphml.gz")

In [ ]:

scglue.data.lsi(atac,n_components=100) # This might take up too much space. If so, preprocess on a node using (xx-glue-preprocessing.py)

In [ ]:
rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")

In [ ]:
atac.write('preprocessed_'+atac_name+'_'+date+".h5ad", compression="gzip")
try:
    rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")
except:
    rna.var = rna.var.drop(columns='artif_dupl')
    rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")
    

In [ ]:
rna

In [ ]:
atac